In [3]:
import pandas as pd
df = pd.read_csv("data\\corona_tested_individuals_ver_006.english.csv")
df.head(4)


C:\Users\daniel.franklin\AppData\Local\Temp\ipykernel_18568\2396665090.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data\\corona_tested_individuals_ver_006.english.csv")


,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication
0,1-30,0,0,0,0,0,0,2,0,0
1,1-30,1,0,0,0,0,0,2,0,0
2,1-30,0,1,0,0,0,0,2,1,0
3,1-30,1,0,0,0,0,0,2,0,0
